# Inference with TRT FP32 model

In [1]:
import time
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants

In [22]:
model = tf.keras.models.load_model('saved_model/mobilenetv2')
print('Converting to TF-TRT FP32...')
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP32,
                                                               max_workspace_size_bytes=8000000000)

converter = trt.TrtGraphConverterV2(input_saved_model_dir='saved_model/mobilenetv2',
                                    conversion_params=conversion_params)

def input_fn():
    for i in range(50):
        inp = tf.random.uniform((1, 160, 160, 3), minval=-1, maxval=1)
        yield [inp]
converter.convert()
converter.build(input_fn=input_fn)
converter.save(output_saved_model_dir='saved_model/mobilenetv2_TFTRT_FP32')
print('Done Converting to TF-TRT FP32')

Converting to TF-TRT FP32...
INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Assets written to: saved_model/mobilenetv2_TFTRT_FP32/assets
Done Converting to TF-TRT FP32


INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Assets written to: saved_model/mobilenetv2_TFTRT_FP32/assets


In [20]:
# !saved_model_cli show --all --dir saved_model/mobilenetv2_TFTRT_FP32

In [17]:
optimized_model = tf.saved_model.load('saved_model/mobilenetv2_TFTRT_FP32', tags=[tag_constants.SERVING])
signature_keys = list(optimized_model.signatures.keys())
print('Signature keys of optimized model: ',signature_keys)
infer = optimized_model.signatures[trt.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
print('Outputs of serving_default: ', infer.structured_outputs)

Signature keys of optimized model:  ['serving_default']
Outputs of serving_default:  {'predictions': TensorSpec(shape=<unknown>, dtype=tf.float32, name='predictions')}


In [18]:
tfds.disable_progress_bar()
ds, metadata = tfds.load(
    'cats_vs_dogs',
    split='train',
    with_info=True,
    as_supervised=True)
get_label_name = metadata.features['label'].int2str
decode_prediction = lambda x: 1 if x>=0 else 0

In [19]:
n_predictions = 0
n_correct_predictions = 0
start_time = time.time()
for image, label in ds.take(1000):
    x = tf.cast(image, tf.float32)
    x = (x/127.5)-1
    x = tf.image.resize(x, (160,160))
    x = tf.expand_dims(x, axis=0)

    preds = infer(x)
    n_predictions += 1
    prediction = preds['predictions'][0,0] # only process first object at first batch index
    decoded_pred = decode_prediction(prediction)
    correct_prediction = label == decoded_pred
    if correct_prediction:
        n_correct_predictions += 1
elapsed_time = time.time() - start_time
accuracy = n_correct_predictions / n_predictions
print('predicteded {} images with accuracy of {:.2f}% with a rate of {:.2f} images/s'.format(n_predictions, accuracy * 100, n_predictions/elapsed_time))

predicteded 1000 images with accuracy of 99.90% with a rate of 401.58 images/s
